<a href="https://colab.research.google.com/github/Rekt77/kisa_insuretech/blob/master/recommandation_webcrawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 한글 형태소 분석기 다운로드
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 3.6MB 41.4MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
# 한국어 형태소분석기
import json
from konlpy.tag import Kkma
kor_nlp = Kkma()

ins_cat = ["운전자","상해","재물",]

# 우리의 불용어 사전
#with open("/content/drive/My Drive/insuretech/nlp_stopword.json","r") as JSON:
#  kor_stopwords=json.load(JSON)["KOR"]

In [17]:
import requests
from bs4 import BeautifulSoup

bobae_list = list()
bobae_test = list()

# 보배드림에서 문장을 긁어 온다 = 운전자
url = "https://www.bobaedream.co.kr/list"
myParams = {
    "code":"national",
    "or_gu":"10",
    "or_se":"desc",
    "pagescale":"30",
    "s_select":"Subject",
    "s_key":"%EB%B3%B4%ED%97%98",#보험
    "type":"list",
    "page": 1
}

res = requests.get(url)

for i in range(1,6):
  myParams["page"]=i
  res = requests.get(url)
  soup = BeautifulSoup(res.content,'lxml')
  titles = soup.find_all("a",class_="bsubject")
  for title in titles:
    if myParams["page"] == 5:
      bobae_test.append([title.text,"운전"])
    else:
      bobae_list.append([title.text,"운전"])


In [18]:
cho_list = list()
cho_test = list()

# 헬스조선에서 문장을 긁어온다 키워드 = 질병
url = 'http://health.chosun.com/list.html'

myParams={
    "nowcode":1,
    "pn":1
}

for i in range(1,6):
  myParams["pn"]=i
  res = requests.get(url,params=myParams)
  soup = BeautifulSoup(res.content,'lxml')
  article_div = soup.find_all("dl",class_="list_item")
  for each in article_div:
    article = each.find_all("a")
    tmp_url = 'http://health.chosun.com/' + article[1].attrs['href']
    res = requests.get(tmp_url)
    soup = BeautifulSoup(res.content,'lxml')
    title = soup.find("h1",id="news_title_text_id")
    if myParams["pn"] == 5:
      cho_test.append([title.text,"질병"])
    else:
      cho_list.append([title.text,"질병"])

In [19]:
dc_list = list()
dc_test = list()

# DC인사이드 보험갤러리 키워드 = 상해
url = "https://gall.dcinside.com/board/lists"

myParams = {"id":"insurance",
            "page":1,
            "s_type":"search_subject",
            "s_keyword":"%EC%83%81%ED%95%B4",#상해
            "board_type":"undefined"}

myHeader = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"
}

for i in range(1,6):
  myParams["page"] = i
  res = requests.get(url,params=myParams,headers=myHeader)
  soup = BeautifulSoup(res.text)
  for each in soup.find_all("td",class_="gall_tit ub-word")[1:]:
    if myParams["page"] == 5:
      print(each.a.text)
      dc_test.append([each.a.text,"상해"])
    else:
      dc_list.append([each.a.text,"상해"])

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


[다나와] 1일 1깡? 난 1일 1쳌하고 에.어.팟.프.로
상해보험 가입하려고 합니다.
상해보험 가입시
디스크 상해가입 받아주는곳
동양생명  무배당수호천사 플러스상해보험이요...
신한은행 참 좋은 상해보험 이거 들었는데..
의료실비보험 추천 100세보장, 비갱신형 암보험, 질병+상해+입원비+암+


In [20]:
joint_list=bobae_list+cho_list+dc_list

In [21]:
test_set = bobae_test+cho_test+dc_test

In [ ]:
test_set

In [23]:
import pickle

with open("webcrawl_joint_data.p","wb") as p:
  pickle.dump(joint_list,p)

with open("webcrawl_joint_data.p","rb") as p:
  joint_list = pickle.load(p)


with open("webcrawl_joint_test.p","wb") as p:
  pickle.dump(test_set,p)

with open("webcrawl_joint_test.p","rb") as p:
  test_set = pickle.load(p)

In [ ]:
test_set

In [26]:
# train_X = 문장 >> ('hi my name is', 'hi your name is')
# train_Y = 분류 >> ('운전','상해')
train_X,train_Y = list(zip(*joint_list))

# 토큰화
# 꼬꼬마 형태소 분석기를 이용해서 문장을 분석
# 형태소 단위로 분해하여 train_x에 저장

train_X = [kor_nlp.morphs(sentence) for sentence in train_X]
print(train_X)

[['오늘', '따라', '유난히', '더', '멋있', '어', '보이', 'ㄴ', '글자'], ['6.25', '70', '주년', '행사', '...'], ['자칭', '보수', '라고', '떠들', '던', 'tv', '조선', '은', '..', 'ㅋㅋㅋ'], ['나', '참', '살', '다', '살', '다', '6.25', '행사', '보', '면서', '감동', '받', '기', '는', '처음', '이', '네'], ['문', '통', '이', '보수', '이', '네요'], ['6.25', '전쟁', '70', '주년', '행사', '드', '론', '퍼포먼스'], ['일본', '신형', '자주포', '클라', '슬'], ['정세', '혀', 'ㄴ', '"', '볼', '턴', '매우', '재수', '없', '는', '사람', '"'], ['일', '베', '가', '또', '등신', '짓', '함', '...'], ['국군', '전사자', '유해', '147', '구', '지금', '내리', '고', '계시', 'ㅂ니다', '.', '실시간'], ['남양', '유업', '회장', '실', '압수', '수색', '...'], ['6.25', '눈물', '이', '나', '네요'], ['대구', '가', '또', '대구', '하', '었', '네'], ['공장', '노동자', '이', 'ㅂ니다', '상여금', '받', '았', '습니다'], ['"', '장관', '말', '듣', '으면', '좋', '게', '지나가', 'ㄹ', '일', '을', '...', '"', '추미애', ',', '윤', '석', '열', '..'], ['한동', '훈', '진짜', '주옥', '되', '었', '군요'], ['편의점', '직원', '폭행', '하', 'ㄴ', '커플', '소식', '..'], ['박상', '학', '선생', '의', '위엄'], ['박', '병석', '에게', '팩스', '보내', 'ㄴ', '용', '자', '분', '등장'], ['

In [ ]:
# import string

# train_X_copy = train_X[:]
# train_X_clense = list()
# # 불용어 제거
# for sentence in train_X_copy:
#   tmp_sentence=list()
#   for token in sentence:
#     if token not in kor_stopwords and token not in string.punctuation:
#       tmp_sentence.append(token)
#   train_X_clense.append(tmp_sentence)

# train_X = train_X_clense[:]

In [ ]:
train_X

In [28]:
# 단어별 인덱스를 만들것임. 모르는 단어는 unknown
# 단어별 인덱스를 만드는 이유 == one-hot-encoding을 위해서
word2index={'unknown' : 0}

# 문장을 추출
for x in train_X:
  # 형태소단위로 분해된 토큰을 추출
    for token in x:
      #word2index에 추가되지 않은 단어이면 단어를 추가
        if word2index.get(token)==None:
          #추가 할때마다 word2index의 길이를 인덱스로 반영
            word2index[token]=len(word2index)

In [29]:
class2index = {'운전' : 0, '질병' : 1, '상해' : 2}
print(word2index)
print(class2index)

{'unknown': 0, '오늘': 1, '따라': 2, '유난히': 3, '더': 4, '멋있': 5, '어': 6, '보이': 7, 'ㄴ': 8, '글자': 9, '6.25': 10, '70': 11, '주년': 12, '행사': 13, '...': 14, '자칭': 15, '보수': 16, '라고': 17, '떠들': 18, '던': 19, 'tv': 20, '조선': 21, '은': 22, '..': 23, 'ㅋㅋㅋ': 24, '나': 25, '참': 26, '살': 27, '다': 28, '보': 29, '면서': 30, '감동': 31, '받': 32, '기': 33, '는': 34, '처음': 35, '이': 36, '네': 37, '문': 38, '통': 39, '네요': 40, '전쟁': 41, '드': 42, '론': 43, '퍼포먼스': 44, '일본': 45, '신형': 46, '자주포': 47, '클라': 48, '슬': 49, '정세': 50, '혀': 51, '"': 52, '볼': 53, '턴': 54, '매우': 55, '재수': 56, '없': 57, '사람': 58, '일': 59, '베': 60, '가': 61, '또': 62, '등신': 63, '짓': 64, '함': 65, '국군': 66, '전사자': 67, '유해': 68, '147': 69, '구': 70, '지금': 71, '내리': 72, '고': 73, '계시': 74, 'ㅂ니다': 75, '.': 76, '실시간': 77, '남양': 78, '유업': 79, '회장': 80, '실': 81, '압수': 82, '수색': 83, '눈물': 84, '대구': 85, '하': 86, '었': 87, '공장': 88, '노동자': 89, '상여금': 90, '았': 91, '습니다': 92, '장관': 93, '말': 94, '듣': 95, '으면': 96, '좋': 97, '게': 98, '지나가': 99, 'ㄹ': 100, '을': 101, '추미애': 102

In [30]:
import torch.nn as nn
import torch
from torch import optim
from torch.autograd import Variable

In [31]:
# Bag of Words 만들기
def make_BoW(seq,word2index):
  #문장에 대한 원핫 인코딩 구현
  #보유한 단어 수 크기의 0으로 초기화된 텐서 구현
    tensor = torch.zeros(len(word2index))

    #seq는 형태소 문장
    #w는 나누어진 형태소
    for w in seq:
      # word2index로 부터 w의 인덱스를 구함
        index = word2index.get(w)
        # 인덱스가 있을 경우
        if index!=None:
          # 해당 인덱스의 값을 1로 만들어줌
            tensor[index]+=1.
        # 인덱스가 없는경우
        else:
          # unknown의 값이 인덱스가 되고
          # 해당 인덱스의 값을 1로 만들어줌
            index = word2index['unknown']
            tensor[index]+=1.
    return tensor

In [32]:
# 문장을 원핫 인코딩으로 만든다.
make_BoW(train_X[0],word2index)

tensor([0., 1., 1.,  ..., 0., 0., 0.])

In [33]:
# ['운전', '을', '배우', '고', '있', '는', '중']의 인덱스가 0,1,2,3,4,5,6 이라면
# 0~6번째 요소가 1로 채워진 텐서가 만들어짐

train_X = torch.cat([Variable(make_BoW(x,word2index)).view(1,-1) for x in train_X])
train_Y = torch.cat([Variable(torch.LongTensor([class2index[y]])) for y in train_Y])

# 각각의 문장이 원핫인코딩 벡터로 나타나져 있는 Variable
train_X

# 각각의 문장에 대응되는 클래스가 나타나져 있는 Variable
train_Y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])

In [34]:
len(train_X)

296

In [35]:
len(train_Y)

296

In [36]:
# BoW classifier 만들기
class BoWClassifier(nn.Module):
    def __init__(self,vocab_size,output_size):
        super(BoWClassifier,self).__init__()
        
        # 뉴런간 선형결합
        # 선형회귀 인풋 차원 : vocab_size
        # 선형회귀 아웃풋 차원 : output_size
        # vocab을 아웃풋에 연결하여 연산
        self.linear = nn.Linear(vocab_size,output_size)
    
    # 아웃풋을 리턴하는 함수 nn.module에서 오버라이팅
    def forward(self,inputs):
        return self.linear(inputs)

In [37]:
def accuracy(pred, y):
    #preds는 데이터가 학습되는 과정에서 라벨을 분류한 값
    preds = torch.argmax(pred, dim=1)
    return (preds == y).float().mean()

# 학습
epochs = 5
LR = 0.1

# 뉴런이 349개
# 인풋 크기는 word2index의 길이
# 아웃풋은 3(클래스가 3개이기 때문)
model = BoWClassifier(len(word2index),3)

# cost함수 선언
# crossentropyLoss
loss_function = nn.CrossEntropyLoss()

# 경사하강법으로 학습
optimizer = optim.SGD(model.parameters(),lr=LR)

for epoch in range(epochs):
    model.zero_grad()
    hypothesis = model(train_X)
    loss = loss_function(hypothesis,train_Y)
    print("epoch:",epoch+1,"loss:",loss.item(),"acc:",float(accuracy(hypothesis,train_Y))*100)
    
    loss.backward()
    optimizer.step()

epoch: 1 loss: 1.0953024625778198 acc: 32.43243098258972
epoch: 2 loss: 1.0729196071624756 acc: 53.04054021835327
epoch: 3 loss: 1.051235556602478 acc: 70.60810923576355
epoch: 4 loss: 1.030231237411499 acc: 78.7162184715271
epoch: 5 loss: 1.0098917484283447 acc: 81.41891956329346


In [41]:
index2class = {v:k for k,v in class2index.items()}

acc = list()
for test in test_set:
    # 테스트 데이터 불러오기
    X = kor_nlp.morphs(test[0])
    X = Variable(make_BoW(X,word2index)).view(1,-1)
    
    pred = model(X)
    
    # 3가지 클래스 분류 중에 가장 높은것 추출
    pred = pred.max(1)[1].item()
    print("Input : %s" % test[0])
    print("Prediction : %s" % index2class[pred])
    print("Truth : %s" % test[1])
    acc.append(index2class[pred]==test[1])

print("Accuracy:",acc.count(True)/len(acc)*100)



Input : 오늘따라 유난히 더 멋있어 보인 글자
Prediction : 운전
Truth : 운전
Input : 6.25 70주년 행사...
Prediction : 운전
Truth : 운전
Input : 자칭 보수라고 떠들던 tv조선은..ㅋㅋㅋ
Prediction : 질병
Truth : 운전
Input : 나참 살다살다 6.25행사보면서 감동받긴처음이네
Prediction : 운전
Truth : 운전
Input : 문통이 보수네요
Prediction : 운전
Truth : 운전
Input : 6.25전쟁 70주년 행사 드론 퍼포먼스
Prediction : 운전
Truth : 운전
Input : 일본 신형 자주포 클라스
Prediction : 질병
Truth : 운전
Input : 정세현 "볼턴 매우 재수 없는 사람"
Prediction : 운전
Truth : 운전
Input : 일베가 또 등신짓 함...
Prediction : 운전
Truth : 운전
Input : 국군 전사자 유해 147구 지금 내리고 계십니다. 실시간
Prediction : 운전
Truth : 운전
Input : 남양유업 회장실 압수수색...
Prediction : 운전
Truth : 운전
Input : 6.25 눈물이 나네요
Prediction : 운전
Truth : 운전
Input : 대구가 또 대구했네
Prediction : 상해
Truth : 운전
Input : 공장 노동자 입니다 상여금 받았습니다
Prediction : 운전
Truth : 운전
Input : "장관 말 들으면 좋게 지나갈 일을..." 추미애, 윤석열 ..
Prediction : 운전
Truth : 운전
Input : 한동훈 진짜 주옥됐군요
Prediction : 상해
Truth : 운전
Input : 편의점 직원 폭행한 커플 소식..
Prediction : 운전
Truth : 운전
Input : 박상학 선생의 위엄
Prediction : 운전
Truth : 운전
Input : 박병석에게 팩스 보낸 용자분 등장
P